In [1]:
## from https://github.com/graykode/nlp-tutorial/tree/master/5-1.Transformer

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import math

def make_batch(sentences):
    input_batch = [[src_vocab[n] for n in sentences[0].split()]]
    output_batch = [[tgt_vocab[n] for n in sentences[1].split()]]
    target_batch = [[tgt_vocab[n] for n in sentences[2].split()]]
    return torch.LongTensor(input_batch), torch.LongTensor(output_batch), torch.LongTensor(target_batch)



## 10
def get_attn_subsequent_mask(seq):
    """
    dec_inputs : [batch_size, tgt_len]
    即seq: [batch_size, tgt_len]
    """
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    # attn_shape: [batch_size, tgt_len, tgt_len]
    # np.triu 它接受一个二维数组（即矩阵）作为输入，并返回该矩阵的上三角部分。
    # k是一个可选参数，用于指定对角线的索引。当k=0时，函数返回主对角线及其上方的元素；
    # 当k为正数时，返回主对角线上方第k条对角线及其上方的元素；
    # 当k为负数时，返回主对角线下方第|k|条对角线及其上方的元素。
    # 生成一个上三角矩阵，其中对角线及其下方的元素为0，对角线以上的元素为1
    subsequence_mask = np.triu(np.ones(attn_shape), k=1)  # 生成一个上三角矩阵
    # subsequence_mask: 
    # [[[0. 1. 1. 1. 1.]
    #   [0. 0. 1. 1. 1.]
    #   [0. 0. 0. 1. 1.]
    #   [0. 0. 0. 0. 1.]
    #   [0. 0. 0. 0. 0.]]]
    # 并将其数据类型转换为byte类型，方便后续进行布尔运算，1是需要屏蔽的位置。
    subsequence_mask = torch.from_numpy(subsequence_mask).byte()
    return subsequence_mask  # [batch_size, tgt_len, tgt_len] 形状是 (1, 5, 5)


## 7. ScaledDotProductAttention
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        ## 输入进来的维度分别是 
        # Q [batch_size, n_heads, len_q, d_k] 
        # K： [batch_size, n_heads, len_k, d_k]  
        # V: [batch_size, n_heads, len_k, d_v]
        # encoder 层 q_s 形状 (1, 8, 5, 64)
        #  k_s 形状 (1, 8, 5, 64)
        #  v_s 形状 (1, 8, 5, 64)
        # attn_mask 形状 (1, 8, 5, 5)

        #  Q*K^T / \sqrt(d_k) 是计算注意力分数
        ##首先经过matmul函数得到的scores形状是 : [batch_size, n_heads, len_q, len_k]
        # scores形状是 (1, 8, 5, 5)
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)

        ## 然后关键词地方来了，下面这个就是用到了我们之前重点讲的attn_mask，
        # 把被mask的地方置为无限小，softmax之后基本就是0，对q的单词不起作用
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        # context = softmax(Q*K^T/\sqrt(d_k)) * V
        
        # nn.Softmax(dim=-1)(scores) 表示在 scores 的最后一个维度len_k上应用 softmax。假设 scores 的形状是 [batch_size, n_heads, len_q, len_k]，
        # 那么 softmax 会在 len_k 这个维度上进行归一化操作。应用 softmax 后，scores 的形状不会改变，仍然是 [batch_size, n_heads, len_q, len_k]，
        # 但其值会被归一化。

        # attn 是 [batch_size, n_heads, len_q, len_k],context 是 [batch_size, n_heads, len_q, d_v]
        # encoder 层 attn 形状 (1, 8, 5, 5)
        # V 形状 (1, 8, 5, 64)
        # 矩阵乘法 对于高于二维的矩阵，第一个矩阵最后一个维度必须和第二个矩阵的倒数第二维度相同
        # 因此context的形状是 (1, 8, 5, 64)
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        # print('attn:',attn.size(),'\nV:',V.size(),'\ncontext:',context.size())
        # attn 形状 (1, 8, 5, 5),context的形状是 (1, 8, 5, 64)

        return context, attn


## 6. MultiHeadAttention
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        ## 输入进来的QKV是相等的，我们会使用映射linear做一个映射得到参数矩阵Wq, Wk,Wv
        # Q,K,V从 [d_model, d_k * n_heads] 形状 (512, 64*8) 理论上这一定成立 d_k * n_heads = d_model，因为我们设置的是8个头，去把d_model拆除d_k=64。
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
        self.linear = nn.Linear(n_heads * d_v, d_model)
        # 对多头进行layernorm，也就是样本归一化
        self.layer_norm = nn.LayerNorm(d_model)

    # Q, K, V 是原始数据，还没乘以对应的权重矩阵WQ,WK,WV
    def forward(self, Q, K, V, attn_mask):

        ## 这个多头分为这几个步骤，首先映射分头，然后计算atten_scores，然后计算atten_value;
        ##输入进来的数据形状： Q: [batch_size , len_q , d_model], K: [batch_size , len_k , d_model], V: [batch_size , len_k , d_model]
        # Encoder层 Q 形状是 (1, 5, 512) K: (1, 5, 512) V: (1, 5, 512)

        # residual 残差，用于后续的残差连接
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)

        ##下面这个就是先映射，后分头；一定要注意的是q和k分头之后维度是一致额，所以一看这里都是dk
        # q_s,k_s,v_s: 真正的Q,K,V矩阵，用来计算注意力分数 
        # 为了将输入分成多个头，每个头独立地进行注意力计算。每个头的维度是 d_k，总共有 n_heads 个头。
        # 通过这种方式，可以并行计算多个注意力头，提高模型的表达能力和计算效率。

        # encoder 层 q_s 形状 (1, 8, 5, 64)
        #  k_s 形状 (1, 8, 5, 64)
        #  v_s 形状 (1, 8, 5, 64)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size  n_heads, len_q, d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size, n_heads, len_k, d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size, n_heads, len_k, d_v]

        ## 原来的attn_mask形状是  [batch_size , len_q , len_k]，
        # encoder层 实际形状是 (1, 5, 5)
        # decoder 层 

        # 新的attn_mask : [batch_size, n_heads, len_q, len_k]，就是把pad信息重复了n个头上
        # encoder 层 attn_mask 形状 (1, 8, 5, 5)
        # decoder 层 

        # reapt函数 :把第二个维度复制n_heads次，也就是把pad信息复制n个头上，其他维度不变
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)


        ##然后我们计算 ScaledDotProductAttention 这个函数，去7.看一下
        ## 得到的结果有两个：context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        # encoder 层 q_s 形状 (1, 8, 5, 64)
        #  k_s 形状 (1, 8, 5, 64)
        #  v_s 形状 (1, 8, 5, 64)
        # attn_mask 形状 (1, 8, 5, 5)
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        # attn 是 [batch_size, n_heads, len_q, len_k]
        # context 是 [batch_size, n_heads, len_q, d_v]
        # encoder 层 attn 形状 (1, 8, 5, 5)
        # 因此context的形状是 (1, 8, 5, 64) 是 softmax(QK^T/\sqrt(d_k)) * V的结果

        ## 下面这个就是把多头的结果拼接起来，然后再映射回去
        # 按照行优先原则，数字在语义和在内存中都是连续的，
        # 当我们使用torch.transpose()方法或者torch.permute()方法对张量翻转后，改变了张量的形状
        # 此时torch.contiguous()方法就派上用场了, 它会将张量中的数据在内存中重新排列, 使得张量变得连续。
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) 
        # context: [batch_size, len_q, n_heads * d_v]
        # encoder 层 context 形状 (1, 5, 512)

        # linear层的意义是把n_heads * d_v 映射回 d_model，但是 n_heads * d_v = d_model
        # 在多头注意力机制中，通常情况下 n_heads * d_v 等于 d_model。即使映射前后的维度没有发生改变，仍然需要进行线性变换。
        # output: [batch_size, len_q, d_model]
        # encoder 层 output 形状 (1, 5, 512)
        output = self.linear(context)
        ## 下面这个就是残差连接，然后进行layernorm
        # attn 是 (1, 8, 5, 5) 
        return self.layer_norm(output + residual), attn # output: [batch_size, len_q, d_model]


## 8. PoswiseFeedForwardNet
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        # 影响从右往左的第二个维度 也就是 in_channels输出为out_channels,
        # 最后一个维度不变，这种Conv1d的操作是对每个单词的维度进行操作
        # 相当于Linear，来实现MLP
        # 输入的维度是 d_model=512 ，输出的维度是 d_ff=2048 ，最后一个维度不变，
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        # 输入的维度是 d_ff=2048 ，输出的维度是 d_model=512 ，最后一个维度不变，
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, inputs):
        # inputs : [batch_size, len_q, d_model]
        # inputs 形状 (1, 5, 512) 
        residual = inputs 
        # self.conv1(inputs.transpose(1, 2)) 后 形状 (1, 2048, 5)
        output = nn.ReLU()(self.conv1(inputs.transpose(1, 2)))
        # output 形状 从(1, 2048, 5) 到 (1, 5, 512)
        output = self.conv2(output).transpose(1, 2)
        # 残差连接后 -> layer_norm 归一化, 
        # 最后形状 (1, 5, 512)
        return self.layer_norm(output + residual)



## 4. get_attn_pad_mask

## 比如说，我现在的句子长度是5，在后面注意力机制的部分，我们在计算出来QK转置除以根号之后，softmax之前，我们得到的形状
## len_input * len*input  代表每个单词对其余包含自己的单词的影响力

## 所以这里我需要有一个同等大小形状的矩阵，告诉我哪个位置是PAD部分，之后在计算计算softmax之前会把这里置为无穷大；

## 一定需要注意的是这里得到的矩阵形状是batch_size, len_q, len_k，我们是对k中的pad符号进行标识，并没有对k中的做标识，因为没必要

## seq_q 和 seq_k 不一定一致，在交互注意力，q来自解码端，k来自编码端，所以告诉模型编码这边pad符号信息就可以，解码端的pad信息在交互注意力层是没有用到的；

def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len] (1, 5)
    seq_k: [batch_size, seq_len] (1, 5)
    '''
    # 自注意力机制: seq_q/ seq_k : [batch_size , src_len]
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token Pad在词表里面编码为0了 
    # 表示对键序列中的填充标记进行掩码。
    # pad_attn_mask 形状 (1,1,5)
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1,len_k] one is masking
#   示例：
#     enc_inputs: tensor([[1, 2, 3, 4, 0]])
#     enc_self_attn_mask: tensor([[[False, False, False, False,  True],
#          [False, False, False, False,  True],
#          [False, False, False, False,  True],
#          [False, False, False, False,  True],
#          [False, False, False, False,  True]]])
#     输出形状 (1,5,5)
# expand函数：在指定的单维度上扩展张量，-1表示不扩展
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size , len_q , len_k]


## 3. PositionalEncoding 代码实现
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()

        ## 位置编码的实现其实很简单，直接对照着公式去敲代码就可以，下面这个代码只是其中一种实现方式；
        ## 从理解来讲，需要注意的就是偶数和奇数在公式上有一个共同部分，我们使用log函数把次方拿下来，方便计算；
        ## pos代表的是单词在句子中的索引，这点需要注意；比如max_len是128个，那么索引就是从0，1，2，...,127
        ##假设我的demodel是512，2i那个符号中i从0取到了255，那么2i对应取值就是0,2,4...510
        self.dropout = nn.Dropout(p=dropout)

        # pe 形状 (5000, 512)
        pe = torch.zeros(max_len, d_model)
        # position 形状 (5000, 1)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # div_term 形状 (256, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)## 这里需要注意的是pe[:, 0::2]这个用法，就是从0开始到最后面，补长为2，其实代表的就是偶数位置
        pe[:, 1::2] = torch.cos(position * div_term)##这里需要注意的是pe[:, 1::2]这个用法，就是从1开始到最后面，补长为2，其实代表的就是奇数位置
        ## 上面代码获取之后得到的pe:[max_len*d_model]

        ## 下面这个代码之后，我们得到的pe形状是：[max_len,1,d_model]
        # torch.unsqueeze(x, 0)#在第0维扩展，第0维大小为1  
        # 下面这个代码之后， pe 形状 (5000, 1, 512)
        pe = pe.unsqueeze(0).transpose(0, 1)

        self.register_buffer('pe', pe)  ## 定一个缓冲区，其实简单理解为这个参数不更新就可以

    def forward(self, x):
        """
        x: [seq_len, batch_size, d_model]
       , 形状 (5, 1, 512)
        """
        # 注意看 pe的实际维度是[max_len, 1, d_model] 也就是 (50000,1,512)
        # 但实际放入模型使用的是 x.size(0) 也就是seq_len，实际的一个句子长度
        x= x + self.pe[:x.size(0), :]
        # self.pe[:x.size(0), :] 中的 :x.size(0) 表示选择第一个维度从 0 到 x.size(0) 的所有元素。
        # : 表示选择第二个维度的所有元素。
        # 第三个维度 d_model 没有显式地写出来，但它仍然保留在结果中。
        # 因此，self.pe[:x.size(0), :] 的结果维度是 [seq_len, 1, d_model]。
        #, 形状 (5, 1, 512)
        return self.dropout(x)


## 5. EncoderLayer ：包含两个部分，多头注意力机制和前馈神经网络
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        ## 下面这个就是做自注意力层，输入是enc_inputs，形状是[batch_size , seq_len_q , d_model] 需要注意的是最初始的QKV矩阵是等同于这个输入的，去看一下enc_self_attn函数 6.
        # enc_inputs, enc_inputs, enc_inputs 最原始的QKV 
        # enc_outputs 是多头注意力机制的输出，每个单词对其他单词的相关性，是 softmax(QK^T/\sqrt(d_k))*V的结果
        #  enc_outputs: [batch_size, len_q, d_model]
        # attn 含义是注意力分数，是softmax(QK^T/\sqrt(d_k))的结果
        # attn: [batch_size, n_heads, len_q, len_k]
        # enc_outputs 形状 (1, 5, 512) 
        # attn 形状 (1, 8, 5, 5) 
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V

        # enc_outputs: [batch_size , len_q , d_model]
        # enc_outputs 形状 (1, 5, 512) 
        enc_outputs = self.pos_ffn(enc_outputs) 
        # 输出enc_outputs 形状 (1, 5, 512)
        # attn 形状 (1, 8, 5, 5) 
        return enc_outputs, attn


## 2. Encoder 部分包含三个部分：词向量embedding，位置编码部分，注意力层及后续的前馈神经网络

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        # 见pytorch学习问题总结
        #(src_vocab_size, d_model) 是 (5, 512)
        self.src_emb = nn.Embedding(src_vocab_size, d_model)  ## 这个其实就是去定义生成一个矩阵，大小是 [src_vocab_size , d_model]
        # 位置编码是把每个词512维的词向量和一个位置编码相加，这个位置编码是一个正余弦函数
        self.pos_emb = PositionalEncoding(d_model) ## 位置编码情况，这里是固定的正余弦函数，也可以使用类似词向量的nn.Embedding获得一个可以更新学习的位置编码
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)]) ## 使用ModuleList对多个encoder进行堆叠，因为后续的encoder并没有使用词向量和位置编码，所以抽离出来；

    def forward(self, enc_inputs):
        ## 这里我们的 enc_inputs 形状是： [batch_size , src_len]
        # enc_inputs 形状 (1, 5)

        ## 下面这个代码通过src_emb，进行索引定位，enc_outputs输出形状是[batch_size, src_len, d_model]
        # 实际上就是把 enc_inputs 从[batch_size , src_len]，加了一个单词被编码的维度 d_model,所以输出结果就是 [batch_size, src_len, d_model]
        # enc_outputs 形状 (1, 5, 512)
        enc_outputs = self.src_emb(enc_inputs)

        ## 这里就是位置编码，把两者相加放入到了这个函数里面，从这里可以去看一下位置编码函数的实现；3.
        # 来回转置操作可能是为了适应 Transformer 模型对输入维度的要求，以及在位置编码之前对输入进行预处理。
        # enc_outputs最后的形状 [batch_size, src_len, d_model]
        # enc_outputs.transpose(0, 1)后 形状 (5, 1, 512)
        # enc_outputs最后的形状 (1, 5, 512)
        
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1)

        ##get_attn_pad_mask是为了得到句子中pad的位置信息，给到模型后面，在计算自注意力和交互注意力的时候去掉pad符号的影响，去看一下这个函数 4.
        #  enc_self_attn_mask: [batch_size , len_q , len_k]

        # enc_inputs, enc_inputs 是 Q,K 形状是 (1, 5)
        # enc_self_attn_mask: [batch_size, len_q, len_k] 形状 (1, 5, 5)
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)
        enc_self_attns = []

        # 堆叠 6 层EncoderLayer
        for layer in self.layers:
            ## 去看EncoderLayer 层函数 5.
            #  enc_outputs [batch_size, src_len, d_model] 形状 (1, 5, 512)
            #  enc_self_attn_mask: [batch_size , len_q , len_k] 形状 (1, 5, 5)
            # layer是EncoderLayer
            # 输出enc_outputs 是 softmax(QK^T/\sqrt(d_k))*V的结果，然后经过MLP，残差连接后layernorm
            # 形状 (1, 5, 512)
            # 注意力分数 softmax(QK^T/\sqrt(d_k))的结果
            #  enc_self_attn 形状 (1, 8, 5, 5) 
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

## 10.
class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.dec_enc_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        # dec_inputs 是decoder词嵌入+位置编码后的结果 [batch_size, tgt_len, d_model]
        # 形状 (1, 5, 512)
        # enc_outputs 是 encoder输出结果 [batch_size, src_len, d_model]
        # 形状 (1, 5, 512)
        # 自注意力 dec_self_attn_mask [batch_size , len_q , len_k] 形状 (1, 5, 5)
        # 交叉注意力 dec_enc_attn_mask [batch_size , len_q , len_k] 形状 (1, 5, 5)

        # 自注意力机制
        # 3个dec_inputs 是原始的Q,K,V [batch_size, tgt_len, d_model] (1, 5, 512)
        # dec_self_attn_mask 是自注意力的mask矩阵，包括pad mask和subsequent mask(掩码矩阵，用于屏蔽未来的信息)
        # dec_self_attn_mask 形状 (1, 5, 5)

        # 输出
        # dec_outputs 是 softmax(QK^T/\sqrt(d_k))*V的结果，然后经过MLP，
        # 残差连接后layernorm [batch_size, tgt_len, d_model] 形状 (1, 5, 512)
        # dec_self_attn 注意力分数 softmax(QK^T/\sqrt(d_k))的结果
        # dec_self_attn 形状 (1, 8, 5, 5)
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)

        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        # de_outputs 是 softmax(QK^T/\sqrt(d_k))*V的结果，然后经过MLP， 残差连接后layernorm
        # dec_outputs 形状 (1, 5, 512)
        # dec_enc_attn 注意力分数 softmax(QK^T/\sqrt(d_k))的结果
        # dec_enc_attn 形状 (1, 8, 5, 5)
        # 下面这个就是前馈神经网络，MLP，残差连接后layernorm,先升维2048再降维512
        dec_outputs = self.pos_ffn(dec_outputs)

        return dec_outputs, dec_self_attn, dec_enc_attn

## 9. Decoder

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        # 加最后一个维度 d_model=512,词映射到向量
        # (tgt_vocab_size, d_model) 是 (7, 512)
        self.tgt_emb = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

    def forward(self, dec_inputs, enc_inputs, enc_outputs): 
        # dec_inputs : [batch_size, tgt_len]
        # enc_inputs : [batch_size, src_len]
        # enc_outputs : [batch_size, src_len, d_model]
        # dec_inputs tensor([[5, 1, 2, 3, 4]]) (1,5) 
        # enc_inputs tensor([[1, 2, 3, 4, 0]]) (1,5)
        # enc_outputs 形状 (1, 5, 512)
        
        # tgt_emb 形状
        # dec_inputs 形状 (1, 5, 512)
        dec_outputs = self.tgt_emb(dec_inputs)  # [batch_size, tgt_len, d_model]
        # 位置编码 形状 (1, 5, 512)
        dec_outputs = self.pos_emb(dec_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, tgt_len, d_model]

        ## 标记 get_attn_pad_mask 自注意力层的时候的pad 部分
        # 实际上就是把Q,K的结果提前预备，标记pad，方便后面多头注意力机制的时候去掉pad的影响
        # dec_self_attn_pad_mask [batch_size , len_q , len_k]
        # dec_self_attn_pad_mask 形状 (1, 5, 5)
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs)

        ## get_attn_subsequent_mask 这个做的是自注意层的mask部分，就是当前单词之后看不到，使用一个上三角为1的矩阵
        # 掩码矩阵，用于屏蔽未来的信息
        # dec_inputs : [batch_size, tgt_len]
        # dec_inputs 形状 (1, 5)，使用维度信息生成mask，维度是  attn_shape: [batch_size, tgt_len, tgt_len] 即 (1, 5, 5)
        dec_self_attn_subsequent_mask = get_attn_subsequent_mask(dec_inputs)

        ## 两个矩阵相加，大于0的为1，不大于0的为0，为1的在之后就会被fill到无限小
        # dec_self_attn_mask [batch_size , len_q , len_k]
        # dec_self_attn_mask 形状 (1, 5, 5)
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequent_mask), 0)


        ## 这个做的是交互注意力机制中的mask矩阵，enc的输入是K，dec输入的Q我去看这个k里面哪些是pad符号，给到后面的模型；
        # 注意哦，我q肯定也是有pad符号，但是这里我不在意的，之前说了好多次了哈

        # dec_inputs : [batch_size, tgt_len]
        # dec_inputs 形状 (1, 5)
        # enc_inputs : [batch_size, src_len]
        # enc_inputs 形状 (1, 5)
        # 输出 [batch_size, tgt_len, src_len] 形状 (1, 5, 5)
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            # 输入部分:
            # dec_outputs 是decoder词嵌入+位置编码后的结果
            # 形状 (1, 5, 512)
            # enc_outputs 是 encoder输出结果 [batch_size, src_len, d_model]
            # 形状 (1, 5, 512)
            # 自注意力 dec_self_attn_mask [batch_size , len_q , len_k] 形状 (1, 5, 5)
            # 交叉注意力 dec_enc_attn_mask [batch_size , len_q , len_k] 形状 (1, 5, 5)

            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        return dec_outputs, dec_self_attns, dec_enc_attns


## 1. 从整体网路结构来看，分为三个部分：编码层，解码层，输出层
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.encoder = Encoder()  ## 编码层
        self.decoder = Decoder()  ## 解码层
        # 映射词表 将d_model 映射为 decoder词表tgt_vocab_size的大小，为了再通过softmax确定到底是词表哪一个，确定输出句子结果
        # d_model, tgt_vocab_size : (512, 7)
        self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False) ## 输出层 d_model 是我们解码层每个token输出的维度大小，之后会做一个 tgt_vocab_size 大小的softmax
    def forward(self, enc_inputs, dec_inputs):
        # enc_inputs/dec_inputs，形状为 batch_size，一个句子长度也就是单词数
        ## 这里有两个数据进行输入，一个是enc_inputs 形状为[batch_size, src_len]，主要是作为编码段的输入，
        # 一个dec_inputs，形状为[batch_size, tgt_len]，主要是作为解码端的输入

        # enc_inputs 形状为 (1, 5), dec_inputs 形状为(1, 5)
        ## enc_inputs作为输入 形状为[batch_size, src_len]，输出由自己的函数内部指定，想要什么指定输出什么，可以是全部tokens的输出，可以是特定每一层的输出；也可以是中间某些参数的输出；
        ## enc_outputs就是主要的输出，是 softmax(QK^T/\sqrt(d_k))*V的结果，然后经过MLP，残差连接后layernorm
        # enc_self_attns这里没记错的是QK转置相乘之后softmax之后的矩阵值，代表的是每个单词和其他单词相关性；
        # enc_outputs 形状 (1, 5, 512)
        # enc_self_attns 形状 (1, 8, 5, 5)
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)

        ## dec_outputs 是decoder主要输出，用于后续的linear映射；
        #  dec_self_attns类比于enc_self_attns 是查看每个单词对decoder中输入的其余单词的相关性；
        # dec_enc_attns是decoder中每个单词对encoder中每个单词的相关性

        # 输入分别是：
        # 一个dec_inputs，形状为[batch_size, tgt_len]，
        # 一个enc_inputs，形状为[batch_size, src_len]，
        # 一个enc_outputs，形状为[batch_size, src_len, d_model]，
        # dec_inputs tensor([[5, 1, 2, 3, 4]]) (1,5)
        # enc_inputs tensor([[1, 2, 3, 4, 0]]) (1,5)
        # enc_outputs 形状 (1, 5, 512)
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)

        ## dec_outputs做映射到词表大小
        dec_logits = self.projection(dec_outputs) # dec_logits : [batch_size, src_vocab_size, tgt_vocab_size]
        return dec_logits.view(-1, dec_logits.size(-1)), enc_self_attns, dec_self_attns, dec_enc_attns



Certainly! Here's a test for the `forward` method of the `Encoder` class:

已进行更改。

## 主要运行部分

In [2]:
## 句子的输入部分，
# sentences 三个分别是 encoder输入，decoder输入，decoder对应的标签
# S 是start ,E 是end ,P是填充字符
sentences = ['ich mochte ein bier P', 'S i want a beer', 'i want a beer E']


# Transformer Parameters
# Padding Should be Zero
## 构建词表
src_vocab = {'P': 0, 'ich': 1, 'mochte': 2, 'ein': 3, 'bier': 4}
# 5
src_vocab_size = len(src_vocab)

tgt_vocab = {'P': 0, 'i': 1, 'want': 2, 'a': 3, 'beer': 4, 'S': 5, 'E': 6}
# 7
tgt_vocab_size = len(tgt_vocab)



# encoder的输入长度
src_len = 5 # length of source
# decoder的输入长度
tgt_len = 5 # length of target

## 模型参数
d_model = 512  # Embedding Size
d_ff = 2048  # FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_layers = 6  # number of Encoder of Decoder Layer
n_heads = 8  # number of heads in Multi-Head Attention

model = Transformer()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

enc_inputs, dec_inputs, target_batch = make_batch(sentences)

# print(enc_inputs, dec_inputs, target_batch)
print(enc_inputs.size())
# enc_inputs : tensor([[1, 2, 3, 4, 0]]) (1,5)
# dec_inputs : tensor([[5, 1, 2, 3, 4]]) (1,5)
# target_batch : tensor([[1, 2, 3, 4, 6]]) (1,5)


for epoch in range(20):
    optimizer.zero_grad()
    outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
    loss = criterion(outputs, target_batch.contiguous().view(-1))
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()





torch.Size([1, 5])
subsequence_mask: [[[0. 1. 1. 1. 1.]
  [0. 0. 1. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]]
Epoch: 0001 cost = 2.233543
subsequence_mask: [[[0. 1. 1. 1. 1.]
  [0. 0. 1. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]]
Epoch: 0002 cost = 1.431044
subsequence_mask: [[[0. 1. 1. 1. 1.]
  [0. 0. 1. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]]
Epoch: 0003 cost = 4.208959
subsequence_mask: [[[0. 1. 1. 1. 1.]
  [0. 0. 1. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]]
Epoch: 0004 cost = 4.000024
subsequence_mask: [[[0. 1. 1. 1. 1.]
  [0. 0. 1. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]]
Epoch: 0005 cost = 5.697386
subsequence_mask: [[[0. 1. 1. 1. 1.]
  [0. 0. 1. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]]
Epoch: 0006 cost = 7.403378
subsequence_mask: [[[0. 1. 1. 1. 1.]
  [0. 0. 1. 1. 1.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]]
Epoch: 0

In [4]:
import torch

# Create an instance of the Encoder class
encoder = Encoder()

# Create dummy input data
enc_inputs = torch.tensor([[1, 2, 3, 4, 0]])  # Shape: (1, 5)

# Call the forward method
enc_outputs = encoder.forward(enc_inputs)

# Print the shape of the output
print(enc_outputs.shape)


tensor([[[[0.2961, 0.5166, 0.2517, 0.6886],
          [0.0740, 0.8665, 0.1366, 0.1025],
          [0.1841, 0.7264, 0.3153, 0.6871]],

         [[0.0756, 0.1966, 0.3164, 0.4017],
          [0.1186, 0.8274, 0.3821, 0.6605],
          [0.8536, 0.5932, 0.6367, 0.9826]]]])
